In [1]:
import json
import pika
import uuid
import threading 
import datetime
import time
import logging
import random
import requests
import os
from enum import Enum
from flask import Flask, request, jsonify
from flask_api import status
from pymessenger import Element, Button
from requests_toolbelt import MultipartEncoder
from pymessenger import utils
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
class NotificationType(Enum):
    regular = "REGULAR"
    silent_push = "SILENT_PUSH"
    no_push = "NO_PUSH"

In [3]:
def verify_fb_token(token_sent,exito):
    VERIFY_TOKEN='bot_oliver'  
    if token_sent == VERIFY_TOKEN:
        return exito
    else:
        return 'Verificacion de token invalida'

In [4]:
def enviarNucleoLectura(recipient_id, id_face, mensaje,hilo): 
    content=''   
    entidad={}
    n =  {"payload": {"robot":"none","id_face":" ","recipient_id":" ","access_token":"none","content":" ","type_resp":"read","state":"none", "blocktype":"none", "opciones":"none", "entities":""}}
    
    data=json.dumps(n)
    decoded=json.loads(data)
    decoded["payload"]["id_face"]=id_face
    decoded["payload"]["recipient_id"]=recipient_id
    decoded["payload"]["content"]=content
    decoded["payload"]["entities"]= entidad  
    
    data_string=json.dumps(decoded)    
    hilo.corr_id = str(uuid.uuid4())
    hilo.channel.basic_consume(on_message_callback=hilo.on_response, auto_ack=True, queue=hilo.queue_name) 
    try:
        hilo.channel.basic_publish(exchange='',
                              routing_key='Colaprueba23',
                              properties=pika.BasicProperties(
                                        reply_to = hilo.queue_name,
                                        correlation_id = hilo.corr_id,
                                        ),
                                body=str(data_string))
               
        print('\n________________________________________read _______________________________________________________________________________')
        print('Mensaje enviado al Nucleo Logico Read-> ',data_string)  
        hilo.start() 
    except Exception as error:
        print("mensaje no enviado...",str(error))

In [5]:
def enviarNucleo(recipient_id, id_face, mensaje,hilo):        
    cont=0
    content=''
    type_resp=''    
    entidad= {}
    if "postback" in mensaje[0]:
        content=mensaje[0]["postback"]["payload"]
        type_resp="postback"
        #entidad={mensaje[0]["postback"]["payload"]}
    elif "message" in mensaje[0]:
        content=mensaje[0]["message"]["text"]
        type_resp="message"    
    estado = "none"
    tipo="none"
    opciones="none"     
    n =  {"payload": {"robot":" ","id_face":" ","recipient_id":" ","access_token":" ","content":" ","type_resp":" ","state":" ", "blocktype":" ", "opciones":" ", "entities":""}}
    data=json.dumps(n)
    decoded=json.loads(data)
    decoded["payload"]["robot"]="none"
    decoded["payload"]["id_face"]=id_face
    decoded["payload"]["access_token"]="none"
    decoded["payload"]["recipient_id"]=recipient_id
    decoded["payload"]["content"]=content
    decoded["payload"]["type_resp"]=type_resp
    decoded["payload"]["state"]=estado
    decoded["payload"]["blocktype"]=tipo
    decoded["payload"]["opciones"]=opciones
    decoded["payload"]["entities"]= entidad
    data_string=json.dumps(decoded)    
    hilo.corr_id = str(uuid.uuid4())
    hilo.channel.basic_consume(on_message_callback=hilo.on_response, auto_ack=True, queue=hilo.queue_name) 
    try:
        hilo.channel.basic_publish(exchange='',
                              routing_key='Colaprueba23',
                              properties=pika.BasicProperties(
                                        reply_to = hilo.queue_name,
                                        correlation_id = hilo.corr_id,
                                        ),
                                body=str(data_string))
        hilo.start()        
        print('\n______________________________________________no read__________________________________________________________________________')
        print('Mensaje enviado al Nucleo Logico-> ',data_string)        
    except Exception as error:
        print("mensaje no enviado...",str(error))

In [6]:
def send_message(payload,access_token):
    api_version = 6.0
    graph_url = 'https://graph.facebook.com/v{0}'.format(api_version)
    request_endpoint = '{0}/me/messages'.format(graph_url)
    auth = {'access_token': access_token}  
    response = requests.post(
        request_endpoint,
        params=auth,
        json=payload
    )
    try:
        result = response.json()
        print("\n---------------------------------------------------------------------------")
        print("mensaje enviado a la API de facebook:\n",result)
        return result
    except:
        print("Error al enviar mensaje")
        return 'fail'


In [7]:
def text_message(recipient_id, response,access_token):
    payload={'message': {'text': response}, 'recipient': {'id': recipient_id}, 'notification_type': 'REGULAR'}
    respuesta=send_message(payload,access_token)
    return "success"

In [8]:
def button_message(recipient_id,content,options,access_token):
    buttons = []
    cont=0
    for opc in options:
        button = Button(title=opc, type='postback', payload=opc)
        buttons.append(button)
        cont=cont+1
        if cont>2:
            break    
    payload={'message': {'attachment': {'type': 'template', 'payload': {'template_type': 'button', 'text': content, 'buttons': buttons}}}, 'recipient': {'id': recipient_id}, 'notification_type': 'REGULAR'}
    result=send_message(payload,access_token)
    assert type(result) is dict
    assert result.get('message_id') is not None
    assert result.get('recipient_id') is not None

In [9]:
def slide_message(recipient_id,content,access_token):
    buttons =[]
    elements=[]
    element={'title':'', 'image_url':'', 'subtitle':'', 'buttons':[]}
    
    cont=0    
    for data in content:         
        buttons =[]
        cont=0 
        for btn in data['buttons']:
            if btn['typeButton']=='web_url':
                button = Button(title=btn['titleButton'], type='web_url', url=btn['urlButton'])                
            elif btn['typeButton']=='postback':
                button = Button(title=btn['titleButton'], type='postback', payload=btn['payloadButton'])
            buttons.append(button)
            cont=cont+1
            if cont>2:
                break
        
        
        element={'title':data['title'], 'image_url':data['image_url'], 'subtitle':data['subtitle'], 'buttons':buttons}
        elements.append(element)   
        
    payload={"recipient": {"id":recipient_id}, "message":{"attachment": {"type":"template", "payload": {"template_type":"generic", "elements":elements}}}, 'notification_type': 'REGULAR'}
    print("pyload slide: ",payload)
    result=send_message(payload,access_token)
    assert type(result) is dict
    assert result.get('message_id') is not None
    assert result.get('recipient_id') is not None

In [10]:
def ticket_message(recipient_id,content,access_token):
    payload={"recipient":{"id":recipient_id},"message":{"attachment":{"type":"template","payload":content}}}    
    result=send_message(payload,access_token)
    assert type(result) is dict
    assert result.get('message_id') is not None
    assert result.get('recipient_id') is not None

In [11]:
def conv(mensaje):
    ans=' '
    contenido=mensaje["Respuesta"]["content"]
    estado=mensaje["Respuesta"]["payload"]["state"]
    entidad= mensaje["Respuesta"]["payload"]["entities"]
    tipo= mensaje["Respuesta"]["blockType"]
    recipient_id=mensaje['Respuesta']['payload']['recipient_id']
    access_token=mensaje['Respuesta']['payload']['access_token']    
    if(tipo=="quickReply" or tipo=="quickReplyDinamico"):
        button_message(recipient_id, contenido, mensaje["Respuesta"]["options"],access_token)
        return entidad, ans, estado, tipo, mensaje["Respuesta"]["options"]        
    elif(tipo=="input" or tipo=="inputDinamico"):
        text_message(recipient_id, contenido,access_token)
        return entidad, ans, estado, tipo, 'vacio'    
    elif(tipo=="informativo" or tipo=="informativoDinamico"):
        text_message(recipient_id, contenido,access_token)
        return entidad, str('next'), estado, tipo, 'vacio'    
    elif(tipo=="slide" or tipo=="slideBuy" or tipo=="slideDinamico"):
        slide_message(recipient_id, contenido,access_token)
        return entidad, str('next'), estado, tipo, 'vacio'
    elif(tipo=="ticket"):
        ticket_message(recipient_id, contenido,access_token)
        return entidad, str('next'), estado, tipo, 'vacio'
    else:
        return entidad, str('hey'), estado, tipo, 'vacio'

In [12]:
class Nucleo_Cliente2(threading.Thread):
    
    def on_response(self, ch, method, props, body):
        if self.corr_id == props.correlation_id:
            self.response = body
            
    def __init__(self, nombreHilo):    
        threading.Thread.__init__(self, name=nombreHilo, target=Nucleo_Cliente2.run)
        self.nombrehilo=nombreHilo
        self.connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
        self.channel = self.connection.channel()
        self.result = self.channel.queue_declare(queue='',exclusive=True)
        self.queue_name = self.result.method.queue
        self.corr_id = ' ' 
        
    def run(self):
        try:                
            self.response = None
            self.response = self.call()
            print(" [.] Got %r", self.response) 
        except Exception as error:
            print("Error run :",str(error))     
            
    def call(self):
        print("hilo consume")
        cont = 0        
        while(cont>=0):
            self.response = None
            while self.response is None:
                self.connection.process_data_events()            
            if len(self.response)>10:
                print(self.response)
                data = json.loads(self.response)                
                if  data.get("payload"):
                    break
                else:                
                    print('\n_________________________________________________________________________________________________________________________')
                    print('Respuesta del Nucleo Logico-> ',data) 
                    entidad, resp, estado, tipo, opciones= conv(data)
                    break
                


In [13]:

#-------------Creación de la REST API con Flask-------------
app = Flask(__name__)
#---------------PASO 1. Recepción de mensajes enviados por la API de Facebook.--------------------------
@app.route("/apiREST/v1", methods=['GET', 'POST'])
def receive_message():
    peticiones = 0
    #---------------PASO 2. Inicio: Identificar el tipo de petición que se está solicitando.--------------------------
    if request.method == 'GET':
        print(" petición GET")
        print("metodo GET...")
        print("request",request)
        token_sent = request.args.get("hub.verify_token")
        print(token_sent )
        #---------------PASO 2.1. Llamada a la función que valida la cadena de verificación y se retorna a la petición GET la validación de la función.--------------------------
        return verify_fb_token(token_sent,request.args.get("hub.challenge"))
    else:
        #---------------Recepción de mensajes de una operacion tipo POST.--------------------------        
        
        content = request.get_json(force=True)
        output = request.get_json()
        print('\n_________________________________________________________________________________________________________________________')
        print("Mesaje de la API de Facebook-> \n",output)    
        #return "Message Processed"  
        if output["object"]=="page":
            for event in output['entry']:
                messaging = event['messaging']
                id_face = event['id']
                for message in messaging:
                    if message.get('read'):
                        nombreHilo="Hilo_"+str(peticiones)
                        N_C = Nucleo_Cliente2(nombreHilo)
                        recipient_id = message['sender']['id'] 
                        enviarNucleoLectura(recipient_id, id_face, messaging,N_C)
                        peticiones=peticiones+1                       
                        
                    elif message.get('message') or message.get('postback'):
                        nombreHilo="Hilo_"+str(peticiones)
                        N_C = Nucleo_Cliente2(nombreHilo)
                        recipient_id = message['sender']['id']                 
                        enviarNucleo(recipient_id, id_face, messaging,N_C)  
                        peticiones=peticiones+1
                    #envio de mensaje proveniente de fb a nucleo lógico del sistema
    
    return "Message Processed"        
#-------------Iniciando la REST API-------------
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Oct/2020 11:45:15] "GET /apiREST/v1?hub.mode=subscribe&hub.challenge=1160904360&hub.verify_token=bot_oliver HTTP/1.1" 200 -


 petición GET
metodo GET...
request <Request 'http://7bcdfce50e9a.ngrok.io/apiREST/v1?hub.mode=subscribe&hub.challenge=1160904360&hub.verify_token=bot_oliver' [GET]>
bot_oliver


127.0.0.1 - - [02/Oct/2020 11:46:15] "POST /apiREST/v1 HTTP/1.1" 200 -



_________________________________________________________________________________________________________________________
Mesaje de la API de Facebook-> 
 {'object': 'page', 'entry': [{'id': '430761350747124', 'time': 1601657175409, 'messaging': [{'sender': {'id': '2566943440020277'}, 'recipient': {'id': '430761350747124'}, 'timestamp': 1601657175214, 'message': {'mid': 'm_fLzdpIP2o_oubnR5P9PsaHQZ8rWMkng5G3qw4TWUTaciJuajxnC4XOGX5IsOfJhhxXtxpQyzqtIFmc4l3rcSQA', 'text': 'hola'}}]}]}
hilo consume

______________________________________________no read__________________________________________________________________________
Mensaje enviado al Nucleo Logico->  {"payload": {"robot": "none", "id_face": "430761350747124", "recipient_id": "2566943440020277", "access_token": "none", "content": "hola", "type_resp": "message", "state": "none", "blocktype": "none", "opciones": "none", "entities": {}}}
b'{"Respuesta": {"content": "\\u00a1Hola! Soy Niza.", "next_id": "next", "blockType": "informativ

...


________________________________________read _______________________________________________________________________________
Mensaje enviado al Nucleo Logico Read->  {"payload": {"robot": "none", "id_face": "430761350747124", "recipient_id": "3812585132088508", "access_token": "none", "content": "", "type_resp": "read", "state": "none", "blocktype": "none", "opciones": "none", "entities": {}}}
hilo consume
b'{"Respuesta": {"content": "\\u00bfQu\\u00e9 te gustar\\u00eda hacer?", "next_id": "next", "blockType": "informativo", "contentType": "text", "typingTime": "3", "payload": {"content": "", "state": "mensajePreMenu_01inicial_inicial", "recipient_id": "3812585132088508", "entities": {"qr_menuIntro": "\\u00a1ok veamos!", "var_pregunta": "\\u00bfqu\\u00e9 edad tienes?", "qr_personalidad": "\\u00bfCon qui\\u00e9n estoy hablando?"}, "access_token": "EAAJjZAQWaAbgBAAxC53jaIzoxSpgY3qZBc0pYDlb5g3E58pTJ35vAzrsZBVSyiYPhpgZB3ZAeNqx18ImA0LZCcsqL96hFDqsE9fcXUk52ofVsqWeerVDKp0cGo2wvohq5ISdkCUX7ZBy

127.0.0.1 - - [02/Oct/2020 13:12:00] "POST /apiREST/v1 HTTP/1.1" 200 -


b'{"Respuesta": {"content": "Selecciona una opci\\u00f3n", "next_id": "mensajesIntroduccion_01inicial_inicial,infoEdad_01inicial_inicial,infoColor_01inicial_inicial", "blockType": "quickReply", "contentType": " ", "typingTime": "3", "payload": {"content": "", "state": "qrPersonalidad_01inicial_inicial", "recipient_id": "3812585132088508", "entities": {"qr_menuIntro": "\\u00a1ok veamos!", "var_pregunta": "\\u00bfqu\\u00e9 edad tienes?", "qr_personalidad": "\\u00bfCon qui\\u00e9n estoy hablando?"}, "access_token": "EAAJjZAQWaAbgBAAxC53jaIzoxSpgY3qZBc0pYDlb5g3E58pTJ35vAzrsZBVSyiYPhpgZB3ZAeNqx18ImA0LZCcsqL96hFDqsE9fcXUk52ofVsqWeerVDKp0cGo2wvohq5ISdkCUX7ZByd4c80Y6BstQYbe5noQhC8L7wdtMuKyZAAgZDZD"}, "options": ["\\u00bfCon qui\\u00e9n estoy hablando?", "\\u00bfQu\\u00e9 edad tienes?", "\\u00bfCu\\u00e1l es tu color favorito?"], "default_id": "NLPpersonalidad_01inicial_inicial", "save_var": "\\u00bfCon qui\\u00e9n estoy hablando?"}}'

___________________________________________________________

127.0.0.1 - - [02/Oct/2020 13:12:18] "POST /apiREST/v1 HTTP/1.1" 200 -



________________________________________read _______________________________________________________________________________
Mensaje enviado al Nucleo Logico Read->  {"payload": {"robot": "none", "id_face": "430761350747124", "recipient_id": "3812585132088508", "access_token": "none", "content": "", "type_resp": "read", "state": "none", "blocktype": "none", "opciones": "none", "entities": {}}}
hilo consume
b'{"payload": {"robot": 340, "id_face": "430761350747124", "recipient_id": "3812585132088508", "access_token": "EAAJjZAQWaAbgBAAxC53jaIzoxSpgY3qZBc0pYDlb5g3E58pTJ35vAzrsZBVSyiYPhpgZB3ZAeNqx18ImA0LZCcsqL96hFDqsE9fcXUk52ofVsqWeerVDKp0cGo2wvohq5ISdkCUX7ZByd4c80Y6BstQYbe5noQhC8L7wdtMuKyZAAgZDZD", "content": "", "type_resp": "read", "state": "none", "blocktype": "none", "opciones": "none", "entities": {}}}'
 [.] Got %r None
